# Ottenimento prossimi match
Lo script deve permettere di prendere i prossimi 10 match. 
Verrà fatta una predizione su questi match, prima deve essere calcolata la media di ciascun match che è data dai 5 match precedenti.

In [18]:
from analysis import MatchAnalysis
import pandas as pd
import util_strings as utils
from ranking import Ranking
from football_predictions import FootballPredictions

tfidf = True

In [19]:
championship = pd.read_csv(utils.championship_actual_season, index_col=0)
season = utils.championship_actual_season.replace('.csv', '')[-9:]

championship['date'] = pd.to_datetime(championship['date'], format='%Y-%m-%d')

championship.result = championship.result.astype(str)
new_matches = championship[championship.result == 'nan']

In [20]:
matches = new_matches[:10][['team1', 'team2', 'date', 'result']]
matches['season'] = season
matches.rename(columns= {'team1': 'home', 'team2': 'away'}, inplace=True)
matches.reset_index(drop=True, inplace=True)
matches.home = matches.home.str.lower()
matches.away = matches.away.str.lower()

matches.to_csv(utils.next_matches)

## Ottenimento testo FootballPrediction
Scaricamento del testo di ciascuna partita

In [21]:
fp = FootballPredictions(matches=matches)
fp.update_originalNotation()
fp.get_urls()
fp.get_predictions(utils.next_matches, False)
fp.df['season'] = season
fp.recovery_games()
fp.matches_not_found()
fp.fix_dates(utils.next_matches)

100%|██████████| 3/3 [00:12<00:00,  4.05s/it]


In [22]:
from my_tokenizer import MyTokenizer

mt = MyTokenizer(fp.df)
mt.feature_normalization()
mt.clean_text()
mt.open_vectorizer(utils.TfidfVectorizer if tfidf else utils.CountVectorizer)

In [23]:
import pickle

dt = pickle.load(open(utils.classificatorTfIdf if tfidf else utils.classificatorCV_LR, 'rb'))
predictions = dt.predict(mt.vectorizer.transform(mt.cleaned_corpus))

Aggiungo le predizioni al dataset dei match

In [24]:
cont = 0
for i, match in matches.iterrows():
    matches.at[i, 'prediction'] = predictions[cont]
    cont += 1

## Calcolo delle medie
Per ciascun match devo calcolare le statistiche date dalle medie dei 5 match precedenti e poi devo ottenere la predizione di football_prediction.
Sostituisco inoltre i codici delle squadre

Per calcolare le medie devo prendere tutti i match di ciascuna squadra

In [25]:
ranking = Ranking('SerieA', '2022-2023')
ranking.read_matches(seasons=6, path=utils.ranking)

ma = MatchAnalysis()
ma.set_ranking(ranking=ranking)
#leggo tutti i match (doppi perché ci sono le statistiche x ogni squadra)
ma.read_matches(utils.merged_statistics)
#creo una lista di oggetti (lista di squadre)
ma.create_team_dataset() 

In [26]:
import json
with open(utils.teams_codes, 'rb') as json_file:
    id_set = json.load(json_file)

In [27]:
matches['date'] = pd.to_datetime(matches['date'], format='%Y-%m-%d') 
float_features_and_avg = ['goals','total_shots','shots_on_target','goals_on_penalty','total_penalties','percentage_possession','fouls','yellow_cards','red_cards']

import numpy as np
for f in float_features_and_avg:
    matches[f] = np.nan

matches[float_features_and_avg] = matches[float_features_and_avg].astype(float)

label = {'N': 0, 'V': 1, 'P':2}

ma.diff_dataset = matches.copy()
for i, match in ma.diff_dataset.iterrows(): 
    match.season = '2022-2023'
    avgH, c = ma.get_team_by_name(match.home).get_avg_last_X_matches(5, match.date, float_features_and_avg)
    avgA, c = ma.get_team_by_name(match.away).get_avg_last_X_matches(5, match.date, float_features_and_avg)
    
    ma.get_ranks(match, i)

    for col in float_features_and_avg:
        diff = avgH[col] - avgA[col]
        ma.diff_dataset.at[i, col] = diff
        
    ma.diff_dataset.at[i, 'home'] = id_set[match.home]
    ma.diff_dataset.at[i, 'away'] = id_set[match.away]

ma.diff_dataset.drop(columns=['result', 'date', 'season'], inplace=True)

for i, m in ma.diff_dataset.iterrows():
    ma.diff_dataset.at[i, 'rank'] = m.rank_h - m.rank_a
ma.diff_dataset.drop(columns=['rank_h', 'rank_a'], inplace=True)

In [28]:
#ma.diff_dataset.home = ma.diff_dataset.home.astype(float)
#ma.diff_dataset.away = ma.diff_dataset.away.astype(float)

In [29]:
ma.diff_dataset
path_model = utils.final_classificator_Tfidf if tfidf else utils.final_classificator_CV
#path_model = 'ML model/Tfidf_SVC.model'
dt = pickle.load(open(path_model, 'rb'))
predictions = dt.predict(ma.diff_dataset)

In [30]:
ma.diff_dataset

,home,away,prediction,goals,total_shots,shots_on_target,goals_on_penalty,total_penalties,percentage_possession,fouls,yellow_cards,red_cards,rank
0,8,16,1.0,1.4,6.2,1.0,0.2,0.2,2.2,-3.0,-1.0,0.0,5.0
1,12,17,1.0,0.6,3.8,2.2,0.2,0.2,6.8,-1.4,-0.6,0.0,4.0
2,25,9,2.0,-0.6,-6.8,-2.6,0.2,0.2,-4.0,1.2,1.6,0.2,-11.0
3,18,11,0.0,-1.6,-2.8,-1.2,0.0,0.0,4.0,2.0,0.6,0.0,-3.0
4,28,5,1.0,-0.4,-3.2,-2.2,-0.2,-0.4,-6.2,-1.0,-0.8,0.0,-11.0
5,22,19,1.0,-0.8,1.6,-1.6,0.2,0.4,-10.6,3.2,-1.6,0.0,-12.0
6,10,27,1.0,2.0,0.4,1.8,0.2,0.4,4.8,-2.2,-0.4,-0.2,12.0
7,4,0,2.0,-0.4,2.4,1.2,-0.2,-0.2,-1.8,0.0,0.0,-0.2,-7.0
8,24,14,2.0,-0.2,-0.6,-0.4,-0.4,-0.4,-0.4,0.8,1.0,0.2,-7.0
9,26,1,0.0,0.4,2.0,2.0,-0.2,-0.2,0.8,1.6,-0.6,0.2,-12.0


In [31]:
df_predictions = pd.DataFrame()
for i, match in matches.iterrows():
    match['prediction'] = predictions[i]
    df_predictions = df_predictions.append(match[['home','away','prediction']])

In [32]:
df_predictions

,home,away,prediction
0,inter,sampdoria,1.0
1,napoli,sassuolo,1.0
2,lecce,juventus,0.0
3,torino,milan,2.0
4,spezia,fiorentina,1.0
5,cremonese,udinese,0.0
6,lazio,salernitana,1.0
7,empoli,atalanta,0.0
8,hellas verona,roma,2.0
9,monza,bologna,2.0
